<a href="https://colab.research.google.com/github/moshe-hadad/knesset-network/blob/main/knessset_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
    <img src="https://en.idi.org.il/media/9162/knesset.jpg" alt="Knesset" />
    <h1>Knesset Network Project - What are we Paying for</h1>
</div>

# Intoroduction
As part of the Social Network Analysis course at the University of Haifa, we are undertaking the Knesset Network Project. <br>
This project aims to explore and reveal the hidden connections between members of the Knesset, connections that are not immediately apparent. Utilizing advanced graph network algorithms, we will analyze the relationships and interactions between these members to uncover patterns and insights into their social networks. Through this analysis, we hope to gain a deeper understanding of the political landscape and the dynamics within the Knesset. <br>

### The first step is to install external libraries for supporting visualizations




In [31]:
# Run this in first run to install pyvis for the graph display
!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00


### Netxt, we set up constants variables for the links to the data sets on GitHub

In [18]:
FACTION_URL ='https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/factions.csv'
FACTION_PEOPLE_URL = 'https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/faction_people.csv'
PERSONS_URL = 'https://raw.githubusercontent.com/moshe-hadad/knesset-network/main/data/KNS_Person_data.csv'


### Now, we remotly loading data sets and presenting some first rows

In [19]:
import pandas as pd
factions_df = pd.read_csv(FACTION_URL)
factions_df.head()

,FactionID,Name,KnessetNum,StartDate,EndDate,IsCurrent,NumberOfMembers,StatusID,StatusDesc,LastUpdatedDate
0,3,ישראל אחת,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
1,4,הליכוד,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
2,5,"ש""ס",15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
3,6,מרצ,15,1999-06-07T00:00:00,NaN,False,NaN,NaN,NaN,2019-01-24T11:45:06.46
4,7,האיחוד הלאומי - ישראל ביתנו,15,2000-02-01T00:00:00,NaN,False,NaN,NaN,NaN,2022-01-23T15:10:00


In [20]:

factions_ppl_df = pd.read_csv(FACTION_PEOPLE_URL)
factions_ppl_df.head()

,PersonID,KnessetNum,FactionID
0,4217,17,638
1,4302,17,634
2,528,16,424
3,526,16,422
4,549,16,52


In [22]:
persons_df = pd.read_csv(PERSONS_URL)
persons_df.set_index('PersonID', inplace=True)
persons_df.head()

,LastName,FirstName,GenderID,GenderDesc,Email,IsCurrent,LastUpdatedDate
PersonID,,,,,,,
48,מלר-הורוביץ,ירדנה,250,נקבה,NaN,False,2015-11-16T09:33:02.47
122,פורדס,גיורא,251,זכר,NaN,False,2020-03-23T23:33:55.387
128,לב,דוד,251,זכר,NaN,False,2020-03-23T23:40:56.46
134,ימין,אלינור,250,נקבה,אלינור ימין,True,2020-11-19T13:43:11.277
136,קפלן,רות,250,נקבה,NaN,False,2020-03-23T23:40:27.3


In [50]:
import networkx as nx
# Function to get person full name for an ID
def person(person_id):
  return f"{persons_df.loc[person_id]['FirstName']} {persons_df.loc[person_id]['LastName']}"

def gender(person_id):
  return persons_df.loc[person_id]['GenderDesc']

G = nx.Graph()

# Function to add a faction
def add_faction(faction_id, name):
    G.add_node(str(faction_id), type='Faction', Name=name, NumberOfMembers=0, label=name)

# Function to add a member and connect to a faction
def add_member(person_id, full_name, gender_desc, faction_id):
  faction_id = str(faction_id)
  person_id = str(person_id)
  if faction_id in G and G.nodes[faction_id]['type'] == 'Faction':
      G.add_node(person_id, type='Member', FullName=full_name,
                 GenderDesc=gender_desc, label=full_name)
      G.add_edge(person_id, faction_id)
      # Update the NumberOfMembers for the faction
      G.nodes[faction_id]['NumberOfMembers'] = G.degree[faction_id]

Creating the Knesset Graph.<br>
In this graph we will create members, factions and which member beongs to which faction.

In [51]:
# Limit the data to the 25th Knesset
factions_df = factions_df[factions_df['KnessetNum'] == 25]
factions_ppl_df = factions_ppl_df[factions_ppl_df['KnessetNum'] == 25]

def knesset_graph():
  # Add factions to the graph from the DataFrame
  for _, row in factions_df.iterrows():
      add_faction(row['FactionID'], row['Name'])

  # Add members to the graph from the DataFrame and connect them to factions
  for _, row in factions_ppl_df.iterrows():
    person_id = row['PersonID']
    add_member(row['PersonID'], person(person_id),gender(person_id),
               row['FactionID'])

  # # Display the graph nodes with attributes
  # for node, data in G.nodes(data=True):
  #     print(f"{node}: {data}")

  return G

In [52]:
from pyvis import network
from IPython.display import display, HTML

# Create a sample graph
G = knesset_graph()

# Create a network object with inline resources
net = network.Network(notebook=True, cdn_resources='remote')

# Load the NetworkX graph into the network object
net.from_nx(G)

# Show the network
net.save_graph("networkx-pyvis.html")
HTML(filename="networkx-pyvis.html")
# net.show("karate_club_graph.html")